In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 9.7 MB/s eta 0:00:00


In [ ]:
import stanza

In [ ]:
stanza.download('en')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nlp = stanza.Pipeline()
stop_words = set(stopwords.words('english'))

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes           |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
txt="freeze is good but compressor is bad"

In [ ]:
def aspect_sentiment_analysis(txt, stop_words, nlp):

    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList=[]
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                  flag=0
                  continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList)


        stop_words = set(stopwords.words('english'))
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)

        doc = nlp(finaltxt) # Object of Stanza Pipeleine

# Initialize a list to store dependency relations
        dep_relations = []

# Iterate through each word in the sentence
        for sent in doc.sentences:
            for word in sent.words:
                governor_index = word.head - 1  # Adjust for zero-based indexing
                if governor_index >= 0:
                    governor_text = sent.words[governor_index].text
                else:
                    governor_text = None  # Handle case when index is less than 0 (e.g., root)

                dependent_text = word.text
                relation_label = word.deprel

                dep_relations.append([dependent_text, governor_text, relation_label])

# Print the extracted dependency relations
#for relation in dep_relations:
 #   print(relation)

        featureList = []
        categoriesList = []

        for i in taggedList:
            if i[1] in ['JJ', 'NN', 'JJR', 'NNS', 'RB']:
                featureList.append(list(i))  # Append [word, tag] to featureList
                categoriesList.append(i[0])  # Append word to categoriesList

        fcluster = []
        for i in featureList:
            filist = []
            for j in dep_relations:
               if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])

        finalcluster = []
        dic = {}
        for i in featureList:
            dic[i[0]] = i[1]
        for i in fcluster:
            if(dic[i[0]]=="NN"):
                finalcluster.append(i)
#print(finalcluster)

        return(finalcluster)




In [ ]:
print(aspect_sentiment_analysis(txt,stop_words,nlp))

[['freeze', ['good']], ['compressor', ['bad']]]


In [ ]:
from google.colab import files
uploaded=files.upload()

KeyboardInterrupt: ignored

In [ ]:
reviews=pd.read_csv('obj_reviews_samsung_refrigerator.csv')

In [ ]:
reviews.head(2)

,Unnamed: 0,Samsung_Refrigerator_343_L3_Obj_reviews
0,0,Wonderful fridge
1,1,storage capacity is good and its also value fo...


In [ ]:
#df.drop(['A'], axis=1)
#reviews.drop(['Unnamed: 0'],axis=1,inplace=True)
# Convert the column to a list
#context_list = df['context_text'].tolist()
reviews_lists = reviews['Samsung_Refrigerator_343_L3_Obj_reviews'].tolist()

In [ ]:
#removing float from reviews
reviews_lists = [x for x in reviews_lists if not isinstance(x, float)]

In [ ]:
#extracting aspect form reviews
aspects = [aspect_sentiment_analysis(txt,stop_words,nlp) for txt in reviews_lists]

In [ ]:

aspects

[[['fridge', ['wonderful']]],
 [['storagecapacity', ['good']],
  ['good', ['storagecapacity']],
  ['value', ['also']],
  ['money', []]],
 [['grillcover', ['back']]],
 [['product', ['overall', 'great']]],
 [['description', ['wrong']]],
 [['fine', ['super', 'works']]],
 [['samsung', ['doorrefrigerator']],
  ['doorrefrigerator', ['samsung', 'double']]],
 [['refrigerator', ['nice']]],
 [['world', []]],
 [['productrefrigerator', []]],
 [['customercare', ['carenumber']], ['carenumber', ['customercare']]],
 [],
 [['refrigerator', ['reasonable']]],
 [['product', ['great']]],
 [['samsung', ['fridges']]],
 [['product', ['amazonpathetic']], ['amazonexperience', ['worst']]],
 [['product', ['good']]],
 [['product', ['fantastic']]],
 [['taptap', ['tapsound']], ['tapsound', ['taptap', 'coming']], ['night', []]],
 [['price', []]],
 [['product', ['damage', 'received']],
  ['customerservice', ['poor', 'samsung']],
  ['support', ['back']],
  ['amazon', []]],
 [['product', ['disappointing']]],
 [['designd

In [ ]:
sentences=aspects

In [ ]:
# Create a dictionary to store related words for each noun
noun_related_words = {}

# Iterate through the sentences
for sentence in sentences:
    for item in sentence:
        noun = item[0]
        attributes = item[1]

        if noun not in noun_related_words:
            noun_related_words[noun] = []

        noun_related_words[noun].extend(attributes)

# Convert the dictionary to the desired output format
output = [[key, value] for key, value in noun_related_words.items()]

# Print the result
for sentence in output:
    print(sentence)


['fridge', ['wonderful', 'switch']]
['storagecapacity', ['good']]
['good', ['storagecapacity']]
['value', ['also']]
['money', []]
['grillcover', ['back']]
['product', ['overall', 'great', 'great', 'amazonpathetic', 'good', 'fantastic', 'damage', 'received', 'disappointing', 'pathetic', 'good', 'damaged', 'got', 'worst']]
['description', ['wrong']]
['fine', ['super', 'works']]
['samsung', ['doorrefrigerator', 'fridges']]
['doorrefrigerator', ['samsung', 'double']]
['refrigerator', ['nice', 'reasonable', 'purchase', 'good']]
['world', []]
['productrefrigerator', []]
['customercare', ['carenumber', 'samsungcustomer']]
['carenumber', ['customercare']]
['amazonexperience', ['worst']]
['taptap', ['tapsound']]
['tapsound', ['taptap', 'coming']]
['night', []]
['price', []]
['customerservice', ['poor', 'samsung']]
['support', ['back']]
['amazon', []]
['designdefect', []]
['service', ['pathetic', 'pthetic']]
['samsungcustomer', ['customercare']]
['switch', ['waiting', 'fridge', 'up']]
['purchase

**Giving points to aspect and then count sentiment score**

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download the VADER lexicon if not already downloaded
nltk.download('vader_lexicon')

# Initialize the Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


### ***function to calculate sentiment scores with custom scoring***

In [ ]:
filtered_sentences=output

In [ ]:

def calculate_sentiment_score(sentence):
    total_score = 0
    for word in sentence:
        sentiment_score = sia.polarity_scores(word)
        compound_score = sentiment_score['compound']

        if compound_score > 0:  # Positive word
            total_score += 2
        elif compound_score < 0:  # Negative word
            total_score -= 2
        # Neutral words have no effect on the score

    return total_score

data = []  # List to store the formatted strings
sentiment_scores = []  # List to store the sentiment scores

# Calculate sentiment scores for each sentence with custom scoring
for noun, words in filtered_sentences:
    sentiment_score = calculate_sentiment_score(words)
    data.append(f'{noun}')
    sentiment_scores.append(sentiment_score)

# Print the formatted strings
#for item in data:
   # print(item)

# Print the sentiment scores
print("Sentiment Scores:", sentiment_scores)

Sentiment Scores: [2, 2, 0, 0, 0, 0, 0, -2, 2, 0, 0, 4, 0, 0, 0, 0, -2, 0, 0, 0, 0, -2, 0, 0, 0, -2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, -4, 0, 0, 2, 0, 0, 0, 0, 0, 0]


In [ ]:
data

['fridge',
 'storagecapacity',
 'good',
 'value',
 'money',
 'grillcover',
 'product',
 'description',
 'fine',
 'samsung',
 'doorrefrigerator',
 'refrigerator',
 'world',
 'productrefrigerator',
 'customercare',
 'carenumber',
 'amazonexperience',
 'taptap',
 'tapsound',
 'night',
 'price',
 'customerservice',
 'support',
 'amazon',
 'designdefect',
 'service',
 'samsungcustomer',
 'switch',
 'purchase',
 'family',
 'item',
 'quality',
 'timedelivery',
 'benefit',
 'costemi',
 'shut',
 'fridgedoor',
 'experience',
 'dent',
 'sidecorner',
 'valuemoney',
 'time',
 'customerfeedback',
 'customer',
 'feedback',
 '..',
 'store']

In [ ]:
df=pd.DataFrame({
    'aspects' : data , 'sentiment_scores' : sentiment_scores
})

In [ ]:
df

,aspects,sentiment_scores
0,product,50
1,warrantyonsite,0
2,buy,4
3,response,0
4,installation,0
5,demo,0
6,use,0
7,😊😊,0
8,awesome,0
9,productoven,0


In [ ]:
df['sentiment_scores'].sum()

2

Product names and there sentiment scores accordingly

In [ ]:
#LG_1.5_Ton_5_Star_AC_Obj_reviews    sentiment_score =  90

In [ ]:
#Samsung_1.5_Ton_5_Star_AC_Obj_reviews    sentiment_score = 22

In [ ]:
#LG_Refrigerator_343_L3_Obj_reviews  sentiment_score =  72


In [ ]:
#Samsung_Refrigerator_343_L3_Obj_reviews  sentiiment_score = 72

In [ ]:
#LG_8_Kg_5_Star_washing_machine_Obj_reviews  sentiment_score = 50

In [ ]:
#Samsung_7_Kg_5_Star_washing_machine_Obj_reviews sentiment_score = 74

In [ ]:
#LG_28_L_oven   sentiment_score = 78
#Samsung_28_L_oven  sentiment_score = 56


In [ ]:
import pandas as pd

# Assuming you already have the 'data' and 'sentiment_scores' lists populated
Lg_Product_names = ['LG_1.5_Ton_5_Star_AC','LG_Refrigerator_343_L3','LG_8_Kg_5_Star_washing_machine','LG_28_L_oven']  # Replace with your actual sentiment scores
Samsung_product_names = ['Samsung_1.5_Ton_5_Star_AC','Samsung_Refrigerator_343_L3','Samsung_7_Kg_5_Star_washing_machine','Samsung_28_L_oven']
Lg_product_sentiment_scores = [90,72,50,78]
samsung_product_sentiment_scores=[22,2,74,56]
# Create a DataFrame
df = pd.DataFrame({'Lg_Product_names': Lg_Product_names , 'Lg_product_sentiment_scores': Lg_product_sentiment_scores , 'samsung_product_sentiment_scores' : samsung_product_sentiment_scores
                   , 'Samsung_product_names' : Samsung_product_names })



In [ ]:
df

,Lg_Product_names,Lg_product_sentiment_scores,samsung_product_sentiment_scores,Samsung_product_names
0,LG_1.5_Ton_5_Star_AC,90,22,Samsung_1.5_Ton_5_Star_AC
1,LG_Refrigerator_343_L3,72,2,Samsung_Refrigerator_343_L3
2,LG_8_Kg_5_Star_washing_machine,50,74,Samsung_7_Kg_5_Star_washing_machine
3,LG_28_L_oven,78,56,Samsung_28_L_oven


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['Lg_product_sentiment_scores'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['samsung_product_sentiment_scores'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_3, *['index'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_14, *['index', 'Lg_product_sentiment_scores', 'Lg_Product_names'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_15, *['index', 'Lg_product_sentiment_scores', 'Samsung_product_names'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_16, *['index', 'samsung_product_sentiment_scores', 'Lg_Product_names'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_17, *['index', 'samsung_product_sentiment_scores', 'Samsung_product_names'], **{})
chart

ERROR:root:Did not find quickchart key chart-9c641ae1-f063-4958-a8d1-a76a296a5793 in chart cache
